<h1><span style="color:gray">ipyrad-analysis toolkit:</span> mrbayes</h1>

In these analyses our interest is primarily in inferring accurate branch lengths under a relaxed molecular clock model. This means that tips are forced to line up at the present (time) but that rates of substitutions are allowed to vary among branches to best explain the variation in the sequence data. 

There is a huge range of models that can be employed using mrbayes by employing different combinations of parameter settings, model definitions, and prior settings. The ipyrad-analysis tool here is intended to make it easy to run such jobs many times (e.g., distributed in parallel) once you have decided on your settings. In addition, we provide a number of pre-set models (e.g., clock_model=2) that may be useful for simple scenarios. 

Here we use simulations to demonstrate the accuracy of branch length estimation when sequences come from a single versus multiple distinct genealogies (e.g., gene tree vs species tree), and show an option to fix the topology to speed up analyses when your only interest is to estimate branch lengths. 

In [2]:
# conda install ipyrad -c conda-forge -c bioconda
# conda install mrbayes -c conda-forge -c bioconda
# conda install ipcoal -c conda-forge

In [3]:
import toytree
import ipcoal
import ipyrad.analysis as ipa

### Simulate a gene tree with 14 tips and MRCA of 1M generations

In [4]:
TREE = toytree.rtree.bdtree(ntips=8, b=0.8, d=0.2, seed=123)
TREE = TREE.mod.node_scale_root_height(1e6)
TREE.draw(ts='o', layout='d', scalebar=True);

<svg class="toyplot-canvas-Canvas" height="260.0px" id="t52da00c136114b76b999d3461a1d5ab7" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r6 r7 0 500000 1000000

### Simulate sequences on single gene tree and write to NEXUS
When Ne is greater the gene tree is more likely to deviate from the species tree topology and branch lengths. By setting recombination rate to 0 there will only be one true underlying genealogy for the gene tree. We set nsamples=2 because we want to simulate diploid individuals. 

In [5]:
# init simulator
model = ipcoal.Model(TREE, Ne=2e4, nsamples=2, recomb=0)

# simulate sequence data on coalescent genealogies
model.sim_loci(nloci=1, nsites=20000)

# write results to database file
model.write_concat_to_nexus(name="mbtest-1", outdir='/tmp', diploid=True)

wrote concat locus (8 x 20000bp) to /tmp/mbtest-1.nex


In [6]:
# the simulated genealogy of haploid alleles
gene = model.df.genealogy[0]

# draw the genealogy
toytree.tree(gene).draw(ts='o', layout='d', scalebar=True);

<svg class="toyplot-canvas-Canvas" height="270.0px" id="tebefa4f2f9714b619eca57434371b013" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 270.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r2-1 r2-0 r1-1 r1-0 r0-1 r0-0 r3-1 r3-0 r4-1 r4-0 r5-1 r5-0 r6-1 r6-0 r7-1 r7-0 0 516272 1032543

### View an example locus
This shows the 2 haploid samples simulated for each tip in the species tree.

In [7]:
model.draw_seqview(idx=0, start=0, end=50);

<svg class="toyplot-canvas-Canvas" height="256.0px" id="t45ea9f2da3174e47b8183f2f2195ebfb" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1020.0 256.0" width="1020.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0-0 r0-1 r1-0 r1-1 r2-0 r2-1 r3-0 r3-1 r4-0 r4-1 r5-0 r5-1 r6-0 r6-1 r7-0 r7-1

### (1) Infer a tree under a relaxed molecular clock model

In [12]:
# init the mb object
mb = ipa.mrbayes(
    data="/tmp/mbtest-1.nex",
    name="itest-1",
    workdir="/tmp",
    clock_model=2,
    constraints=TREE,
    ngen=int(1e6),
    nruns=2,
)

# modify a parameter
mb.params.clockratepr = "normal(0.01,0.005)"
mb.params.samplefreq = 5000

# summary of priors/params
print(mb.params)

brlenspr      clock:uniform       
clockratepr   normal(0.01,0.005)  
clockvarpr    igr                 
igrvarpr      exp(10.0)           
nchains       4                   
ngen          1000000             
nruns         2                   
samplefreq    5000                
topologypr    fixed(fixedtree)    



In [13]:
# start the run
mb.run(force=True)

job itest-1 finished successfully


In [15]:
# load the inferred tree
mbtre = toytree.tree("/tmp/itest-1.nex.con.tre", 10)

# scale root node to 1e6
mbtre = mbtre.mod.node_scale_root_height(1e6)

# draw inferred tree 
c, a, m = mbtre.draw(ts='o', layout='d', scalebar=True);

# draw TRUE tree in orange on the same axes
TREE.draw(
    axes=a, 
    ts='o', 
    layout='d', 
    scalebar=True, 
    edge_colors="darkorange",
    node_sizes=0,
    fixed_order=mbtre.get_tip_labels(),
);

<svg class="toyplot-canvas-Canvas" height="260.0px" id="t365630c82ece43aabed9d52cd1c0e74a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r1 r0 r2 r3 r4 r5 r7 r6 r0 r1 r2 r3 r4 r5 r6 r7 0 500000 1000000

In [16]:
# check convergence statistics
mb.convergence_stats

,Mean,Variance,Lower,Upper,Median,minESS,avgESS,PSRF
Parameter,,,,,,,,
TH,1.057e-02,1.270e-06,8.548e-03,1.280e-02,1.058e-02,96.759,123.880,0.997
TL,3.975e-02,1.433e-05,3.405e-02,4.663e-02,3.930e-02,93.967,122.484,0.997
r(A<->C),1.711e-01,1.907e-04,1.438e-01,1.966e-01,1.694e-01,46.516,98.758,0.998
r(A<->G),1.618e-01,1.587e-04,1.335e-01,1.835e-01,1.633e-01,82.391,116.695,0.999
r(A<->T),1.646e-01,1.955e-04,1.436e-01,1.969e-01,1.626e-01,21.969,86.484,1.022
r(C<->G),1.522e-01,1.635e-04,1.286e-01,1.748e-01,1.528e-01,7.775,79.388,1.012
r(C<->T),1.920e-01,1.813e-04,1.660e-01,2.139e-01,1.928e-01,21.517,73.623,1.023
r(G<->T),1.582e-01,1.961e-04,1.355e-01,1.874e-01,1.568e-01,23.112,87.056,1.012
pi(A),2.473e-01,6.721e-06,2.422e-01,2.526e-01,2.477e-01,137.517,144.259,0.998


### (2) Concatenated sequences from a species tree
Here we use concatenated sequence data from 100 loci where each represents one or more distinct genealogies. In addition, Ne is increased to 1e5, allowing for more genealogical variation. *We expect the accuracy of estimated edge lengths will decrease since we are not adequately modeling the genealogical variation when using concatenation.* Here I set the recombination rate *within* loci to be zero. There is free recombination among loci, however, since they are unlinked.

In [17]:
# init simulator
model = ipcoal.Model(TREE, Ne=1e5, nsamples=2, recomb=0)

# simulate sequence data on coalescent genealogies
model.sim_loci(nloci=100, nsites=200)

# write results to database file
model.write_concat_to_nexus(name="mbtest-2", outdir='/tmp', diploid=True)

wrote concat locus (8 x 20000bp) to /tmp/mbtest-2.nex


In [25]:
# the simulated genealogies of haploid alleles
genes = model.df.genealogy[:4]

# draw the genealogies of the first four loci
toytree.mtree(genes).draw(ts='o', layout='r', height=250);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t2a6bd76e141d49709c51bfb446d81620" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 560.0 250.0" width="560.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r2-1 r2-0 r0-1 r1-1 r1-0 r0-0 r3-1 r3-0 r4-1 r4-0 r5-1 r5-0 r6-1 r6-0 r7-1 r7-0 r3-1 r3-0 r4-1 r4-0 r5-1 r5-0 r1-1 r1-0 r0-1 r2-0 r2-1 r0-0 r7-1 r7-0 r6-1 r6-0 r1-0 r0-0 r0-1 r1-1 r2-1 r2-0 r3-1 r3-0 r5-1 r5-0 r4-1 r4-0 r6-1 r6-0 r7-1 r7-0 r4-1 r4-0 r3-1 r3-0 r1-1 r1-0 r0-1 r5-1 r5-0 r2-1 r0-0 r2-0 r6-1 r6-0 r7-1 r7-0

In [26]:
# init the mb object
mb = ipa.mrbayes(
    data="/tmp/mbtest-2.nex",
    workdir="/tmp",
    name="itest-2",
    clock_model=2,
    constraints=TREE,
    ngen=int(1e6),
    nruns=2,
)

# summary of priors/params
print(mb.params)

# start the run
mb.run(force=True)

brlenspr      clock:uniform       
clockratepr   normal(0.01,0.005)  
clockvarpr    igr                 
igrvarpr      exp(10.0)           
nchains       4                   
ngen          1000000             
nruns         2                   
samplefreq    1000                
topologypr    fixed(fixedtree)    

job itest-2 finished successfully


In [27]:
# load the inferred tree
mbtre = toytree.tree("/tmp/itest-2.nex.con.tre", 10)

# scale root node from unitless to 1e6
mbtre = mbtre.mod.node_scale_root_height(1e6)

# draw inferred tree 
c, a, m = mbtre.draw(ts='o', layout='d', scalebar=True);

# draw true tree in orange on the same axes
TREE.draw(
    axes=a, 
    ts='o', 
    layout='d', 
    scalebar=True, 
    edge_colors="darkorange",
    node_sizes=0,
    fixed_order=mbtre.get_tip_labels(),
);

<svg class="toyplot-canvas-Canvas" height="260.0px" id="t0827bec0002344b0b7c47993d9b3ed32" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r1 r0 r2 r3 r4 r5 r7 r6 r0 r1 r2 r3 r4 r5 r6 r7 0 500000 1000000

In [28]:
mb.convergence_stats

,Mean,Variance,Lower,Upper,Median,minESS,avgESS,PSRF
Parameter,,,,,,,,
TH,1.371e-02,9.286e-07,1.189e-02,1.567e-02,1.371e-02,536.255,571.212,0.999
TL,5.898e-02,1.207e-05,5.240e-02,6.553e-02,5.870e-02,490.961,540.829,0.999
r(A<->C),1.496e-01,1.543e-04,1.236e-01,1.715e-01,1.498e-01,510.420,543.507,1.000
r(A<->G),1.325e-01,1.460e-04,1.105e-01,1.569e-01,1.317e-01,588.878,616.804,1.000
r(A<->T),2.180e-01,2.215e-04,1.871e-01,2.453e-01,2.173e-01,592.980,618.963,1.000
r(C<->G),2.347e-01,2.366e-04,2.023e-01,2.616e-01,2.346e-01,549.871,584.678,1.001
r(C<->T),1.403e-01,1.450e-04,1.191e-01,1.653e-01,1.403e-01,543.051,607.157,1.001
r(G<->T),1.248e-01,1.276e-04,1.047e-01,1.485e-01,1.246e-01,535.020,643.010,1.000
pi(A),2.498e-01,9.551e-06,2.432e-01,2.555e-01,2.498e-01,396.019,481.191,0.999


### To see the NEXUS file (data, parameters, priors):

In [30]:
mb.print_nexus_string()

#NEXUS

[log block]
log start filename=/tmp/itest-2.nex.log replace;

[data block]
execute /tmp/mbtest-2.nex;

[tree block]
begin trees;
  tree fixedtree = ((r7,r6),(r5,(r4,(r3,(r2,(r1,r0))))));
end;

[mb block]
begin mrbayes;
  set autoclose=yes nowarn=yes;

  lset nst=6 rates=gamma;

  prset brlenspr=clock:uniform;
  prset clockvarpr=igr;
  prset igrvarpr=exp(10.0);
  prset clockratepr=normal(0.01,0.005);
  prset topologypr=fixed(fixedtree);

  

  mcmcp ngen=1000000 nrun=2 nchains=4;
  mcmcp relburnin=yes burninfrac=0.25;
  mcmcp samplefreq=1000;
  mcmcp printfreq=10000 diagnfr=5000;
  mcmcp filename=/tmp/itest-2.nex;
  mcmc;

  sump filename=/tmp/itest-2.nex;
  sumt filename=/tmp/itest-2.nex contype=allcompat;
end;

[log block]
log stop filename=/tmp/itest-2.nex.log append;



### (3) Tree inference (not fixed topology) and plotting support values
Here we will try to infer the topology from a concatenated data set (i.e., not set a constraint on the topology). I increased the ngen setting since the MCMC chain takes longer to converge when searching over topology space. Take note that the support values from mrbayes newick files are available in the "prob{percent}" feature, as shown below. 

In [31]:
# init the mb object
mb = ipa.mrbayes(
    data="/tmp/mbtest-2.nex",
    name="itest-3",
    workdir="/tmp",
    clock_model=2,
    ngen=int(2e6),
    nruns=2,
)

# summary of priors/params
print(mb.params)

# start run
mb.run(force=True)

brlenspr      clock:uniform       
clockratepr   normal(0.01,0.005)  
clockvarpr    igr                 
igrvarpr      exp(10.0)           
nchains       4                   
ngen          2000000             
nruns         2                   
samplefreq    1000                
topologypr    uniform             

job itest-3 finished successfully


The tree topology was correctly inferred 

In [32]:
# load the inferred tree
mbtre = toytree.tree("/tmp/itest-3.nex.con.tre", 10)

# scale root node from unitless to 1e6
mbtre = mbtre.mod.node_scale_root_height(1e6)

# draw inferred tree 
c, a, m = mbtre.draw(
    layout='d',
    scalebar=True, 
    node_sizes=18, 
    node_labels="prob{percent}",
);

<svg class="toyplot-canvas-Canvas" height="260.0px" id="t0c1f63754fd947e6b30eea2cc68a0945" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 100 100 100 100 100 100 r1 r0 r2 r3 r4 r5 r7 r6 0 500000 1000000

The branch lengths are not very accurate in this case:

In [33]:
# load the inferred tree
mbtre = toytree.tree("/tmp/itest-3.nex.con.tre", 10)

# scale root node from unitless to 1e6
mbtre = mbtre.mod.node_scale_root_height(1e6)

# draw inferred tree 
c, a, m = mbtre.draw(ts='o', layout='d', scalebar=True);

# draw true tree in orange on the same axes
TREE.draw(
    axes=a, 
    ts='o', 
    layout='d', 
    scalebar=True, 
    edge_colors="darkorange",
    node_sizes=0,
    fixed_order=mbtre.get_tip_labels(),
);

<svg class="toyplot-canvas-Canvas" height="260.0px" id="t6502e8d0db7c45d2a0492f38a4408fc2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r1 r0 r2 r3 r4 r5 r7 r6 r0 r1 r2 r3 r4 r5 r6 r7 0 500000 1000000